In [1]:
from bs4 import BeautifulSoup as soup
import requests

In [2]:
base_url = 'https://www.zillow.com/'
states = [
    'co','in', 'nc', 'sc', 'ga', 'fl', 'tx', 'ny', 'ca', 'nj', 'oh', 'ky'
]
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}


In [3]:
import pandas as pd
basic_df = pd.DataFrame(columns=['zpid', 'bed', 'bath', 'sqft', 'price'])
attr_df = pd.DataFrame(columns=['Type', 'Year Built', 'Heating', 'Cooling',
                       'Parking', 'Lot', 'Price/sqft', 'Buyers Agency Fee', 'zpid'])
inter_df = pd.DataFrame(columns=['duration', 'views', 'saves'])
addr_df = pd.DataFrame(
    columns=['streetAddress', 'postalCode', 'latitude', 'longitude', 'state', 'zpid'])

attr_names = ['Type', 'Year Built', 'Heating', 'Cooling',
              'Parking', 'Lot', 'Price/sqft', 'Buyers Agency Fee']


In [4]:
def crawl_detail(link, zpid):
    try:
        # driver = webdriver.Chrome(
        #     '/media/oem/DATA/20221/chromedriver_linux64/chromedriver')
        # driver.get(link)
        # see_more_texts = ['See complete price history', 'See complete tax history']
        # buttons = [button for button in driver.find_elements(By.TAG_NAME, "button") if button.get_attribute(
        #     'class') == 'StyledTextButton-c11n-8-73-0__sc-n1gfmh-0 iYsTeY']
        # for button in buttons:
        #     try:
        #         span = button.find_element(By.TAG_NAME, 'span')
        #         if span.text in see_more_texts:
        #             button.click()
        #     except:
        #         print('')
        # page = driver.page_source
        # bsobj_0 = soup(page, 'html.parser')
        print(link)
        
        page = requests.get(url=link, headers=headers)
        bsobj_0 = soup(page.content, 'html.parser')
        price = bsobj_0.find('span', {'data-testid': 'price'}).text
        
        attrs = {}
        for attr_name in attr_names:
            title = bsobj_0.find('title', text=attr_name)
            attrs[attr_name] = title.findParent('span').find_next_sibling(
                'span').text if title != None else None
        attrs['zpid'] = zpid
        values = []
        for (key, value) in attrs.items():
            values.append(value)

        [bed, bath, sqft] = [item.findChild().text for item in bsobj_0.find_all(
            'span', {'data-testid': 'bed-bath-item'})]

        iteraction_dl = bsobj_0.find(
            'dl', class_='hdp__sc-7d6bsa-0 cUSEtc')
        interactions = iteraction_dl.find_all(
            'dt') if iteraction_dl != None else None
        [duration, views, saves] = [it.findChild(
        ).text for it in interactions] if interactions != None else [None, None, None]

        
        # price_table = bsobj_0.find('table', class_='hdp__sc-f00yqe-2 cQFnnD')
        # if price_table != None:

        #     histories = [tr.findChildren('td') for tr in price_table.findChildren(
        #         'tr', class_='hdp__sc-f00yqe-3 kURlOl') if tr != None]

        #     for history in histories:
        #         if history == None:
        #             continue
        #         date = history[0].text
        #         status = history[1].text
        #         price = history[2].text.split()[0]
        #         price_hist_df.loc[len(price_hist_df.index)] = [
        #             date, status, price, zpid]

        # tax_table = bsobj_0.find('table', class_='hdp__sc-f00yqe-2 kkEhCu')
        # tax_body = tax_table.find('tbody') if tax_table != None else None
        # if (tax_body != None):
        #     for tr in tax_body.findChildren('tr'):
        #         if tr == None:
        #             continue
        #         tds = tr.findChildren('td')
        #         year = tds[0].text
        #         property_tax = tds[1].text.split()[0]
        #         tax_assessment = tds[2].text.split()[0]
        #         tax_hist_df.loc[len(tax_hist_df.index)] = [
        #             year, property_tax, tax_assessment, zpid]

        # monthly_table = bsobj_0.find('div', class_='sc-jOxtWs dLdXue')
        # print(monthly_table)
        
        # monthlies = [button.find('span').text for button in monthly_table.find_all('button')] if monthly_table != None else None
        # if monthlies != None:
        #     monthlies.append(zpid)


        # ul = bsobj_0.find('ul', id='ds-nearby-schools-list')
        # lis = ul.find_all('li') if ul != None else None
        # schools = []
        # if lis != None:
        #     for li in lis:
        #         schools.append(li.find('span').text)
        

        
        basic_df.loc[len(basic_df.index)] = [zpid, bed, bath, sqft, price]

        attr_df.loc[len(attr_df.index)] = values

        inter_df.loc[len(inter_df.index)] = [duration, views, saves]

        # monthly_df.loc[len(monthly_df.index)] = monthlies
        # schools_df.loc[len(schools_df.index)] = [schools, zpid]
    except:
        print("Exception occured")

In [5]:
import json


def crawl_page(state, page_num):
    url = base_url + state + '/' + str(page_num) + '_p'
    print(url)
    page = requests.get(url=url, headers=headers)
    bsobj = soup(page.content, 'html.parser')
    lis = bsobj.find_all(
        'li', class_='ListItem-c11n-8-73-8__sc-10e22w8-0 srp__hpnp3q-0 enEXBq with_constellation')
    scrs = [li.find('script') for li in lis] if lis != None else None
    if scrs != None:
        for scr in scrs:
            if scr == None:
                continue
            try:
                aDict = json.loads(scr.text)
                link = aDict['url']
                if link.split('/')[3] != 'homedetails':
                    continue
                zpid = link.split('/')[5].split('_')[0]
                streetAddress = aDict['address']['streetAddress']
                postalCode = aDict['address']['postalCode']
                latitude = aDict['geo']['latitude']
                longitude = aDict['geo']['longitude']
                addr_df.loc[len(addr_df.index)] = [streetAddress,
                                                   postalCode, latitude, longitude, state, zpid]
                crawl_detail(link, zpid)
            except:
                print("Exception occured")


In [6]:

for state in states:
    for count in range(1, 21):
        try:
            crawl_page(state, count)
        except:
            print("Exception occured")


https://www.zillow.com/co/1_p
https://www.zillow.com/homedetails/9539-Clatsop-Dr-Colorado-Springs-CO-80925/247509225_zpid/
https://www.zillow.com/homedetails/1722-N-Chelton-Rd-Colorado-Springs-CO-80909/13602165_zpid/
https://www.zillow.com/homedetails/6530-Burrows-Rd-Colorado-Springs-CO-80908/13535937_zpid/
Exception occured
https://www.zillow.com/homedetails/18202-E-Nassau-Dr-Aurora-CO-80013/13097640_zpid/
https://www.zillow.com/homedetails/8495-Rain-Dance-Ct-Colorado-Springs-CO-80920/13561330_zpid/
https://www.zillow.com/homedetails/10520-Romblon-Way-Northglenn-CO-80234/12967801_zpid/
https://www.zillow.com/homedetails/292-S-Salem-Ct-Aurora-CO-80012/13040426_zpid/
https://www.zillow.com/homedetails/2689-S-Patton-Ct-Denver-CO-80219/13383661_zpid/
https://www.zillow.com/co/2_p
https://www.zillow.com/homedetails/215-S-Julian-St-Denver-CO-80219/13342434_zpid/
https://www.zillow.com/homedetails/7605-Marmot-Poin-Colorado-Springs-CO-80922/2062540395_zpid/
https://www.zillow.com/homedetails/

In [ ]:
basic_df.to_csv('data/basic.csv')
attr_df.to_csv('data/attributes.csv')
inter_df.to_csv('data/interactions.csv')
addr_df.to_csv('data/address_grades.csv')